In [1]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
from datetime import *
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event,text
import pyodbc
import requests
import inspect
from validate import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from send_email import *
import re
# set to_email
to_email = ['nthieu@savills.com.vn','pthihuongnguyen@savills.com.vn']
# to_email = ['nthieu@savills.com.vn']

### CONNECT TO AZURE SQL

In [2]:
import json
f = open ('database_information.json', "r")
qq = json.loads(f.read())
f.close()
ini_cnt_str ='Driver={driver_str};Server=tcp:hkazdevsqld3vnreserch.database.windows.net,1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
quoted = quote_plus(ini_cnt_str)
cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(cnt_str)

#Test Connection
try:
    conn = engine.connect()
    result = conn.execute(text("SELECT 1"))
    print("CONNECTION SUCESSFUL!")
except Exception as e:
    print("CONNECTION FAILED:",str(e))

CONNECTION SUCESSFUL!


### CONNECT TO SHAREPOINT

In [6]:
def ConnectSharePoint(url_hub):
    header_Hub = f"share_point_{url_hub.split('/')[2].replace('-','')}"
    config_Hub = read_config_json(config_path, header_Hub)
    Hub = SharePoint(config_Hub)
    #Hub.check_connect()
    return Hub

### READ AND LIST FOLDER

In [8]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = ConnectSharePoint(relative_url).get_content_url(relative_url,return_list_folder=True)

Folder name: /sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Tien Giang Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Nam Dinh Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Vinh Phuc Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Hai Duong Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Binh Duong Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Quang Nam Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Ninh Binh Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Thai Nguyen Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Nghe An Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Dong Nai Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Quang Ngai Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Da Nang City
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Thai Binh Province
/sit

In [9]:

def GetProvinceIS(provinces, list_folder):  
        selected_provinces = []
        province_list = []
        for i in list_folder:
            province_list.append(i.split('/')[7])
        for input_province in provinces:
            input_province = input_province.strip().lower()
            if input_province == "-1":
                break

            matched_provinces = [province for province in province_list if input_province in province.lower()]
            if matched_provinces:
                selected_provinces.extend(matched_provinces)
            else:
                print(f"Không tìm thấy tỉnh chứa từ khóa '{input_province}'.")
        return selected_provinces 

In [18]:

str= input(f"Nhap chuoi cac file muon import: ")
# Remove punctuation after ".xlsx" and before sector (IP or Macro)
str = re.sub(r'(?<=\.xlsx)[^;]+?(?=IP|Macro)', '', str)

# Add semicolon (;) after each ".xlsx" extension
str = re.sub(r'(?<=\.xlsx)', ';', str)

# Replace multiple consecutive semicolons with a single semicolon
str = re.sub(r';+', ';', str)

# Remove any trailing semicolon at the end of the string
str = str.rstrip(';')
print(str)
# # Split the string by semicolon
# file_names = str.split(';')
# sectors = []
# provinces = []
# dates = []
# for file_name in file_names:
# # Remove leading/trailing whitespaces
#     file_name = file_name.strip()

#     # Extract sector, province, and date
#     sector, province, date_ext = file_name.split('_')

#     # Extract date without extension
#     date = os.path.splitext(date_ext)[0]

#     # Append values to respective arrays
#     if (sector == 'Macro'):
#         sectors.append(f'{sector} economic')
#     else: 
#         sectors.append(sector)
#     provinces.append(province)
#     dates.append(date)
# quarters = []
# years = []
# for date in dates:
#     year = int(date[:4])
#     month = int(date[4:6])
#     day = int(date[6:])
#     years.append(year)
#     # Determine the last day of the quarter
#     if month in [3]:
#         quarter = 'Q1'
#     elif month in [6]:
#         quarter = 'Q2'
#     elif month in [9]:
#         quarter = 'Q3'
#     else:
#         quarter = 'Q4'
#     quarters.append(quarter)
# selected_year_quarter = []
# url_hub = []
# Hub = []
# list_folder =[]
# for item in range (len(sectors)):
#     url_hub.append(f'/sites/BIHub/Shared Documents/Advisory Data/{sectors[item]}/Flat file')
#     Hub.append(ConnectSharePoint(url_hub[item]))
#     list_folder.append(Hub[item].get_content_url(url_hub[item], return_list_folder=True))
#     selected_provinces = GetProvinceIS(provinces, list_folder[item])
#     folder_sub2 = f"{url_hub[item]}/{selected_provinces[item]}/{years[item]}/{quarters[item]}"
#     selected_year_quarter.append(folder_sub2)
# df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
# for i, url in enumerate(selected_year_quarter):
#     df_summ_file = pd.concat([df_summ_file, Hub[i].get_content_url(url)])
# list_url = df_summ_file['ServerRelativeUrl'].to_list()
# url = []
# for i in list_url:
#     sector = i.split('/')[-1].split('_')[0].upper()
#     if sector not in ('x'):
#         url.append(i)
#     else:
#         pass


IP_Bac Giang_20171231.xlsx;IP_Bac Giang_20181231.xlsx;IP_Bac Giang_20191231.xlsx;IP_Bac Giang_20201231.xlsx;IP_Bac Giang_20211231.xlsx;IP_Bac Giang_20220630.xlsx;IP_Bac Ninh_20220331.xlsx;IP_Bac Ninh_20230331.xlsx;IP_Bac Ninh_20201231.xlsx;IP_Bac Ninh_20220930.xlsx; IP_Bac Ninh_20220630.xlsx; IP_Bac Ninh_20220331.xlsx;IP_Bac Ninh_20191231.xlsx


In [13]:
quarters = []
years = []
for date in dates:
    year = int(date[:4])
    month = int(date[4:6])
    day = int(date[6:])
    years.append(year)
    # Determine the last day of the quarter
    if month in [3]:
        quarter = 'Q1'
    elif month in [6]:
        quarter = 'Q2'
    elif month in [9]:
        quarter = 'Q3'
    else:
        quarter = 'Q4'
    quarters.append(quarter)
print(years, quarters)

NameError: name 'dates' is not defined

In [ ]:
selected_year_quarter = []
for item in range (len(sectors)):
    url_hub=f'/sites/BIHub/Shared Documents/Advisory Data/{sectors[item]}/Flat file' 
    folder_sub2 = f"{url_hub}/{selected_provinces[item]}/{years[item]}/{quarters[item]}"
    selected_year_quarter.append(folder_sub2)
    print(selected_year_quarter[item])
    # In danh sách các folder đã tìm thấy
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in selected_year_quarter:
    df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])
list_url = df_summ_file['ServerRelativeUrl'].to_list()


/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Bac Ninh Province/2023/Q1
Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Bac Ninh Province/2023/Q1/IP_BacNinh_20230331.xlsx


### GET IMPORT FILE

In [ ]:
url = []
for i in list_url:
    sector = i.split('/')[-1].split('_')[0].upper()
    if sector not in ('x'):
        url.append(i)
    else:
        pass
url

['/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Bac Ninh Province/2023/Q1/IP_BacNinh_20230331.xlsx']

In [ ]:
columns_that_need_unidecode=['Project_Name', 'Sub_Project_Name', 'Developer_Name'
                             , 'City', 'District', 'Target_Industry']
print(list_folder, url_hub, list_url, to_email, columns_that_need_unidecode,cnt_str, engine, sp_object,df_summ_file, Hub)

[['/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Tien Giang Province', '/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Nam Dinh Province', '/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Vinh Phuc Province', '/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Hai Duong Province', '/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Binh Duong Province', '/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Quang Nam Province', '/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Ninh Binh Province', '/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Thai Nguyen Province', '/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Nghe An Province', '/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Dong Nai Province', '/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Quang Ngai Province', '/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Da Nang City', '/sites/BIHub/Shared Documents/Advisory Data/IP/Flat fi

### IMPORT DATA

In [ ]:
'''Prepare ingredients''' 
columns_that_need_unidecode=['Project_Name', 'Sub_Project_Name', 'Developer_Name'
                             , 'City', 'District', 'Target_Industry']
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
#Create multi empty df for tracking autdit step
name_sector = ['IP']
name_sector = [x.lower() for x in name_sector]
for i in name_sector:
    globals()['df_temp_flat_{}'.format(i)] = pd.DataFrame([])
    globals()['df_flat_{}'.format(i)] = pd.DataFrame([])
    globals()['df_new_key_{}'.format(i)] = pd.DataFrame([])  
#-------------------------------------------------------
'''Get data''' 
print('Start Get Data...')
for file_url in tqdm(url):  
    data, file_name, sector = get_data(relative_url, file_url)
    #-------------------------------------------------------
    data = check_date_key(file_url, data)#Check format date_key in flat file   
    data['Project_Name']= np.where(data['Project_Name'].isnull(), data['Sub_Project_Name'], data['Project_Name'])#Fill up project_name if its null
    data['Project_Sub_Type']= np.where(data['Project_Sub_Type'].isnull(), data['Project_Type'], data['Project_Sub_Type'])#Fill up Sub_Type if its null
    
    #Check duplicate sub_name
    print('Start Check duplicate sub_name...')
    data, df_dup = check_duplicate(data, 'Sub_Project_Name')
    if len(df_dup) != 0:
        print(colored('Check duplicate sub_name', 'yellow'))
        df_noti_html = convert_df_to_html(type_html = 1, df = df_dup, type_sector = 2, cnxn = engine)
        run_email(type_sector = 'IP', email_type = 1, user_email = to_email, df_noti_html = df_noti_html)
    else:       
        #-------------------------------------------------------
        
        '''Validation step'''
        print('Start validation step...')
        #Remove unfortmated values
        data = remove_unformated_character(data)
        #Remove unicode characters
        for i in columns_that_need_unidecode:
            data[i] = remove_unicode(data[i])
        #Check dictionary
        print('Start Check dictionary...')
        lst_dict = ['City', 'District', 'Status', 'Sub_Type', 'Type']
        lst_cls = ['City', 'District', 'Status', 'Project_Sub_Type', 'Project_Type']
        for i, j in zip(lst_cls, lst_dict):
            print(f'Start checking dictionary of [{i}:{j}]...')
            data, df_dict = check_dictionary(df_dict, file_name, data, i, j, sector, engine, sp_object)
        print(f'End Check dictionary with df_dict = {len(df_dict)} ...')
        
        if len(df_dict) == 0:
            print(colored('Validate succesfully','green'))
            #-------------------------------------------------------
            '''Import data process'''
            #Check project key
            data = Generate_Additional_Columns(data,url,df_summ_file,BIHub,engine,file_url)
            processed_data, flag_key = check_project_key(file_url, data, sector, engine)
            df_temp_flat_ip = pd.concat([df_temp_flat_ip, data], axis=0)
            df_flat_ip = tracking_flat_file(df_temp_flat_ip, file_url)
            if len(processed_data) != 0:
                df_new_key_ip = check_new_key(df_new_key = df_new_key_ip, processed_data = processed_data, sector = sector)
            #Get key and generate new key (if needed)
            data = get_project_key(flag_key, processed_data, data, sector, engine)
            # insert_to_fresh(file_url, data, cnt_str)
        
            
            #Test insert_to_fresh
            try:
                result = insert_to_fresh(file_url, data, cnt_str)
                print(colored("insert_to_fresh SUCESSFUL!",'green'))
            except Exception as e:
                print(colored("insert_to_fresh FAILED:",str(e),'red'))
            
        else:
            print(df_dict)
            print(colored('Validate failed','red'))
            pass

Start Get Data...


  0%|          | 0/1 [00:00<?, ?it/s]

More than two 'Date_Key' in IP_BacNinh_20230331
Start Check duplicate sub_name...
Start validation step...
Start Check dictionary...
Start checking dictionary of [City:City]...
Start checking dictionary of [District:District]...
Start checking dictionary of [Status:Status]...
Start checking dictionary of [Project_Sub_Type:Sub_Type]...
Start checking dictionary of [Project_Type:Type]...
End Check dictionary with df_dict = 0 ...
Validate succesfully
CONNECTION SUCESSFUL IN INSERT TO FRESH FUNCTION!
Insert IP_BacNinh_20230331 to Fresh
Insert IP_BacNinh_20230331 to Fresh
Insert IP_BacNinh_20230331 to Fresh


100%|██████████| 1/1 [00:18<00:00, 18.94s/it]

insert_to_fresh SUCESSFUL!


### SEND EMAIL

In [ ]:
list_df_flat = [df_flat_ip]
list_df_new_key = [df_new_key_ip]
#Email notify missing in dictionary
if len(df_dict) != 0:
    print(colored('Missing values in dictionary','yellow'))
    df_noti_html = convert_df_to_html(type_html = 2, df = df_dict, cnxn = engine)
    run_email(email_type = 2, user_email = to_email, df_noti_html = df_noti_html)
else:
    pass

#Email notify imported data
df_flat_html, df_query_html = convert_df_to_html(type_html = 3, list_df = list_df_flat, type_sector = 2, cnxn = engine)
run_email(type_sector = 'IP', email_type = 3, user_email = to_email, df_flat_html = df_flat_html, df_query_html = df_query_html)

#Email notify create new key
df_new_key_html = convert_df_to_html(type_html = 4, list_df = list_df_new_key, cnxn = engine)
if len(df_new_key_html) != 0:
    print(colored('Some new keys were created','yellow'))
    run_email(type_sector = 'IP', email_type = 4, user_email = to_email, df_noti_html = df_new_key_html)

In [ ]:
df_dict

### TRACKING AUDIT

In [ ]:
insert_to_tracking(list_df_flat, 'Tracking_IP', engine)